In [1]:
from pandas import Series, DataFrame
import config
import mysql.connector
from datetime import datetime
from mysql.connector import errorcode

In [2]:
#SQL credentials. Store in config file.

sql_host = config.sql_host
sql_user = config.sql_user
sql_password = config.sql_password

In [3]:
# Returns tuple for each sql row.
def selectlast(granularity, time, rows):
    
    sql = ("SELECT * FROM (SELECT * FROM {} WHERE time <= '{}' "
           "ORDER BY time DESC LIMIT {})a ORDER BY time ASC;".format(granularity, time, rows))
    cnx = mysql.connector.connect(user=sql_user, password=sql_password,
                             host=sql_host, database='USDJPY')
    cursor = cnx.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    cursor.close()
    cnx.close()
    return results;

In [4]:
# Create dataframe from sql query result
def dataframe(sqltable):
    names = ['time', 'openAsk', 'closeAsk', 'lowAsk', 'highAsk', 'openBid', 'closeBid', 'lowBid', 'highBid', 'volume']
    df = DataFrame(sqltable, columns = names)
    df.set_index('time', inplace=True)
    return df

In [5]:
# Adds simple moving average to dataframe
def add_sma(df, val):
    df['SMA{}'.format(val)] = df['openBid'].rolling(window=val).mean()

In [6]:
df = dataframe(selectlast('H3', datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'), 20))
add_sma(df, 3)
add_sma(df, 10)
df

,openAsk,closeAsk,lowAsk,highAsk,openBid,closeBid,lowBid,highBid,volume,SMA3,SMA10
time,,,,,,,,,,,
2017-02-08 10:00:00,112.342,112.021,111.993,112.392,112.327,112.007,111.978,112.377,4284,NaN,NaN
2017-02-08 13:00:00,112.022,111.769,111.639,112.063,112.007,111.755,111.625,112.049,12750,NaN,NaN
2017-02-08 16:00:00,111.767,112.160,111.673,112.228,111.753,112.146,111.658,112.212,9155,112.029000,NaN
2017-02-08 19:00:00,112.164,111.944,111.913,112.187,112.148,111.921,111.895,112.175,3272,111.969333,NaN
2017-02-08 22:00:00,111.993,112.013,111.758,112.115,111.912,111.997,111.717,112.092,5456,111.937667,NaN
2017-02-09 01:00:00,112.014,112.115,112.014,112.353,111.996,112.101,111.995,112.329,5125,112.018667,NaN
2017-02-09 04:00:00,112.115,112.294,112.093,112.311,112.100,112.277,112.079,112.297,3382,112.002667,NaN
2017-02-09 07:00:00,112.295,112.248,112.164,112.436,112.277,112.234,112.150,112.421,5837,112.124333,NaN
2017-02-09 10:00:00,112.248,112.364,112.224,112.481,112.235,112.350,112.211,112.467,3731,112.204000,NaN


In [ ]:
# Use for analysis
def add_state(df):
    state = 'Above'
    for i in df.index:
        if df.ix[i, 'SMA3'] < df.ix[i, 'SMA10']:
            state = 'Below'
        else:
            state = 'Above'

        df.ix[i, 'State'] = state

In [ ]:
# Use for analysis
def rearrange_cols(df):
    return df[['time', 'openAsk', 'lowAsk', 'highAsk', 'closeAsk', 'openBid', 'lowBid', 'highBid',
             'closeBid', 'SMA3', 'SMA10', 'State',]]

In [ ]:
# change to work with json data

def crosses(df):
    crosses = 0

    for i in df.index[20:]:
        if check_trend(df, i):
            if df.ix[i-1,'State'] == 'Below' and df.ix[i, 'State'] == 'Above' : # True when SMA3 crosses over SMA10
                diff = ((df.ix[i, 'SMA10'] - df.ix[i-15, 'SMA10']) / df.ix[i, 'SMA10']) * 100
                spread = (df.ix[i, 'openAsk'] - df.ix[i, 'openBid']) *100
                print('{} at {} {}, Spread: {}'.format(
                        round(diff, 6), df.ix[i, 'time'],df.ix[i, 'date'], round(spread ,9)))
                crosses += 1

    print('Crossed above {} times.'.format(crosses))